## Script Recording with Checksum Validation

### What This Does

This system records Python scripts to a text file with integrity protection. It ensures the script hasn't been tampered with before execution.

### How?

1. Take Python script as text
2. Calculate SHA-256 hash (checksum)
3. Base64-encode the script
4. Save: checksum + newline + encoded_script
5. To run: decode, verify checksum, execute if valid

### Validation Process
- **Read**: Load checksum and encoded script from file
- **Decode**: Convert base64 back to original Python code
- **Verify**: Calculate checksum of decoded script
- **Compare**: Check if calculated checksum matches stored checksum
- **Execute**: Only run script if checksums match (integrity confirmed)

In [1]:
import hashlib
import base64
import ast
import subprocess
import tempfile
import os

def startScriptRecording(script_text):
    script_bytes = script_text.encode('utf-8')
    checksum = hashlib.sha256(script_bytes).hexdigest()
    encoded_script = base64.b64encode(script_bytes).decode('utf-8')
    return encoded_script, checksum

def writeScriptWithChecksum(filename, encoded_script, checksum):
    with open(filename, 'w') as f:
        f.write(checksum + '\n')
        f.write(encoded_script + '\n')
    print(f"✓ Saved to {filename}: checksum + encoded script")

def readScriptWithChecksum(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    checksum = lines[0].strip()
    encoded_script = ''.join(line.strip() for line in lines[1:])
    print(f"✓ Loaded from {filename}: checksum + encoded script")
    return encoded_script, checksum

def validateScriptAST(script_text):
    """
    Parse and validate AST to ensure only safe operations are allowed.
    Returns True if script contains only allowed node types.
    """
    try:
        tree = ast.parse(script_text)
        # Define allowed AST node types for basic operations
        allowed_nodes = {
            ast.Module, ast.Expr, ast.Call, ast.Name, ast.Load, ast.Store,
            ast.Constant, ast.Str, ast.Num, ast.BinOp, ast.UnaryOp,
            ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Mod, ast.Pow,
            ast.Compare, ast.Eq, ast.NotEq, ast.Lt, ast.LtE, ast.Gt, ast.GtE,
            ast.Assign, ast.AugAssign, ast.FunctionDef, ast.Return,
            ast.For, ast.While, ast.If, ast.Pass, ast.Import, ast.ImportFrom
        }
        
        for node in ast.walk(tree):
            if type(node) not in allowed_nodes:
                print(f"✗ Unsafe AST node detected: {type(node).__name__}")
                return False
        return True
    except SyntaxError as e:
        print(f"✗ Syntax error in script: {e}")
        return False

def validateAndRunScript(encoded_script, checksum):
    """
    TESTING ONLY: Validate and execute script with security checks.
    WARNING: This pattern is for testing purposes only and must not be used in production.
    """
    try:
        # Decode base64 payload only once
        script_bytes = base64.b64decode(encoded_script.encode('utf-8'))
        script_text = script_bytes.decode('utf-8')
        
        # Calculate checksum of the decoded script
        calculated_checksum = hashlib.sha256(script_bytes).hexdigest()
        
        # Verify checksum matches
        if calculated_checksum != checksum:
            print("✗ Invalid: checksum mismatch")
            return False
        
        # Validate AST for safety (basic check)
        if not validateScriptAST(script_text):
            print("✗ Invalid: script contains unsafe operations")
            return False
        
        # Safer execution approach: run in isolated subprocess
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
            temp_file.write(script_text)
            temp_file.flush()
            
            try:
                # Execute in subprocess with limited privileges
                result = subprocess.run(
                    ['python', temp_file.name],
                    capture_output=True,
                    text=True,
                    timeout=10,  # Prevent infinite loops
                    cwd=tempfile.gettempdir()
                )
                
                if result.returncode == 0:
                    print(f"✓ Valid & executed safely")
                    if result.stdout.strip():
                        print(f"Output: {result.stdout.strip()}")
                    return True
                else:
                    print(f"✗ Script execution failed: {result.stderr}")
                    return False
                    
            except subprocess.TimeoutExpired:
                print("✗ Script execution timed out")
                return False
            except Exception as e:
                print(f"✗ Execution error: {e}")
                return False
            finally:
                # Clean up temporary file
                try:
                    os.unlink(temp_file.name)
                except OSError:
                    pass
                    
    except (base64.binascii.Error, UnicodeDecodeError) as e:
        print(f"✗ Decoding error: {e}")
        return False
    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        return False

# Example usage
script = 'print("Hello from recorded script;")'
print(f"Script: {script}")

encoded_script, checksum = startScriptRecording(script)
print(f"Checksum: {checksum}")

# Write checksum and script to one file
writeScriptWithChecksum('recorded_script_with_checksum.txt', encoded_script, checksum)

# Read and validate from the same file
loaded_script, loaded_checksum = readScriptWithChecksum('recorded_script_with_checksum.txt')

validateAndRunScript(loaded_script, loaded_checksum)

Script: print("Hello from recorded script;")
Checksum: 22d808df71c2ae3534eaace31581c4b42170399d82ff6c2c5fcfb26227e4f51e
✓ Saved to recorded_script_with_checksum.txt: checksum + encoded script
✓ Loaded from recorded_script_with_checksum.txt: checksum + encoded script
✓ Valid & executed safely
Output: Hello from recorded script;


True